# Qwen (通义千问) 模型推理实战

本笔记本演示了如何使用 HuggingFace `transformers` 库加载并运行 Qwen 系列模型进行对话生成。

### 核心步骤：
1. **环境配置**：设置镜像加速与库导入。
2. **加载分词器与模型**：深入理解词表（Vocabulary）与预训练权重。
3. **对话模板格式化**：应用 Chat Template 以符合模型训练时的协议。
4. **推理生成**：将文本转化为张量，通过模型矩阵运算生成答案。

In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from dotenv import load_dotenv

# 1. 环境变量配置
# 从项目根目录的 .env 文件中加载配置（如 HF_TOKEN, HF_ENDPOINT）
# 类比 C++ 中的配置文件读取，这避免了将密钥硬编码在源码中。
'''
自动鉴权：transformers 库的 from_pretrained 方法在运行时会自动检查 os.environ["HF_TOKEN"]。
只要 load_dotenv 成功运行，模型加载时就会自动带上你的 Token。
'''
load_dotenv(dotenv_path="../../.env")

# 2. 指定模型 ID
model_id = "Qwen/Qwen1.5-0.5B-Chat"

/root/workspace/hello-agents/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 3. 加载分词器 (Tokenizer)
# 加载词表 (Vocabulary) 和合并规则 (BPE Merges)。

tokenizer = AutoTokenizer.from_pretrained(model_id)

# 4. 加载模型权重
# 设置设备：优先使用 GPU (CUDA)，否则使用 CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"使用计算设备: {device}")

# 加载预训练模型并移动到指定设备 (类似把代码加载进 RAM/VRAM)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
print("模型加载完成！")

使用计算设备: cpu


Loading weights: 100%|██████████| 291/291 [00:00<00:00, 849.30it/s, Materializing param=model.norm.weight]                              
The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


模型加载完成！


In [3]:
# 5. 对话准备与模板格式化
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "你好，请介绍你自己。"}
]

# 将结构化对话序列化为模型可接受的原始文本协议 (ChatML 格式)
# 为何需要这一步？
# 因为大模型本质上是一个“接龙”机器。
# 如果你直接把“你好”传给它，它可能不知道这是用户问出的问题。
# 通过这个模板，我们把对话框里的聊天记录“翻译”成了模型在训练阶段见过的、标准化的原始长文本，
# 确保它能正确区分子系统背景、用户指令和它自己的回答。

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

print("格式化后的输入文本:")
print(text)

# 6. 编码输入并转换为 Tensor (类似 C++ 的二进制输入流)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

格式化后的输入文本:
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
你好，请介绍你自己。<|im_end|>
<|im_start|>assistant



In [4]:
# 7. 模型推理生成
# max_new_tokens 限定了生成的长度。这就像接龙，模型每次预测下一个数字直到停止。
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)

# 8. 后处理：截断输入部分并解码为文本
# 模型输出包含了输入文本的 ID，我们需要切片除去前面的输入部分。
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

# 将 ID 转换回人类可读的文字，并跳过特殊标记
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("\n模型的回答:")
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



模型的回答:
我是来自阿里云的超大规模语言模型，我叫通义千问。我有着强大的自然语言处理能力，可以回答问题、创作文字，还能表达观点、撰写代码。我与你有什么事情吗？
